# Hamming weight extractor

- Extracts the Hamming weight of the state after the SBox in the first round.
- Appends a hamming weight column to the ASCAD file.

In [1]:
import h5py
from src.aes.main import AES
from src.data.loaders.ascad import ASCADData, ASCAD_DATA, ASCAD_DATA_VAR, ASCADDataType, ASCAD_DATA_FIX
from tqdm import tqdm
import numpy as np

ascad = ASCADData.raw()
plaintext_ix = 0
full_key_ix = 1

META = ascad["metadata"]

def get_acc(metadata):
    return np.zeros((len(metadata), 16), dtype=np.uint8)

KEY, PLAIN = get_acc(META), get_acc(META)

for ix in tqdm(range(len(META))):
    KEY[ix] = META[ix][full_key_ix]
    PLAIN[ix] = META[ix][plaintext_ix]

ascad.close()

100%|██████████| 300001/300001 [01:16<00:00, 3905.33it/s]


In [2]:
from src.aes.hw_sbox import hw_sbox_round1

def gen_hws(plaintext, key):
    """
    Extracts the hamming weight of the state bytes, after the SBox of the first round of AES.
    """
    hws = np.zeros((len(plaintext), 10, 16), dtype=np.uint8)

    # noinspection PyShadowingNames
    for ix in tqdm(range(len(plaintext))):
        aes = AES(key[ix])
        aes.encrypt(plaintext[ix])

        hws[ix] = aes.hw_after_round

    return hws

HWS = gen_hws(PLAIN, KEY)

100%|██████████| 300001/300001 [01:48<00:00, 2752.61it/s]


In [3]:
np.mean(HWS)

4.000081937226876

In [4]:
KEY[:, 0].shape

(300001,)

In [5]:
wrt = h5py.File(f"{ASCAD_DATA}/ASCAD_data/ASCAD_databases/atmega8515-raw-traces.h5", 'r+')

if "hamming_weights" in wrt:
    del wrt["hamming_weights"]
wrt["hamming_weights"] = HWS
wrt["key0"] = KEY[:, 0]

wrt.close()

chk = h5py.File(f"{ASCAD_DATA}/ASCAD_data/ASCAD_databases/atmega8515-raw-traces.h5", 'r')
print(np.mean(chk["hamming_weights"]))
print(list(chk["key0"]))
chk.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[177, 39, 219, 25, 54, 123, 117, 11, 126, 30, 158, 163, 161, 236, 57, 142, 200, 149, 62, 249, 173, 234, 190, 78, 55, 202, 82, 1, 41, 29, 74, 136, 167, 105, 190, 197, 104, 71, 166, 59, 75, 36, 122, 60, 219, 232, 99, 201, 60, 148, 47, 247, 171, 159, 28, 88, 90, 60, 86, 54, 136, 242, 35, 148, 235, 159, 128, 196, 64, 93, 85, 178, 68, 115, 152, 175, 230, 11, 175, 193, 113, 104, 63, 99, 92, 224, 36, 122, 254, 50, 192, 56, 31, 106, 189, 177, 198, 57, 165, 157, 32, 124, 101, 135, 207, 30, 159, 83, 1, 34, 6, 73, 12, 237, 137, 231, 115, 8, 247, 102, 3, 51, 235, 161, 45, 191, 69, 116, 241, 179, 50, 236, 212, 32, 207, 16, 51, 116, 66, 28, 29, 128, 106, 246, 40, 238, 171, 199, 48, 254, 140, 108, 43, 196, 104, 47, 75, 23, 251, 181, 74, 143, 237, 15, 114, 234, 179, 57, 81, 91, 165, 15, 152, 118, 231, 150, 64, 62, 51, 122, 193, 31, 101, 22, 137, 7, 103, 78, 190, 225, 233, 93, 86, 160, 81, 246, 195, 176, 37, 15, 238, 206, 128, 175, 165, 83, 246, 218, 124, 72, 188, 14, 248, 207, 239, 236, 17, 14, 208, 6

In [6]:
ascad = ASCADData.raw()
ascad["hamming_weights"][0]

array([[5, 5, 3, 6, 7, 2, 6, 6, 3, 1, 5, 5, 3, 5, 3, 5],
       [4, 5, 3, 5, 3, 5, 6, 6, 4, 3, 5, 4, 4, 4, 4, 2],
       [5, 3, 4, 4, 5, 2, 5, 5, 6, 7, 4, 4, 4, 3, 4, 4],
       [4, 4, 2, 3, 4, 5, 3, 2, 5, 6, 5, 7, 5, 4, 5, 2],
       [4, 6, 3, 5, 3, 6, 3, 4, 5, 6, 6, 4, 2, 5, 4, 1],
       [5, 3, 4, 5, 6, 3, 5, 3, 5, 2, 6, 1, 5, 4, 3, 6],
       [6, 3, 4, 4, 4, 5, 5, 2, 5, 5, 4, 4, 3, 6, 4, 3],
       [3, 4, 3, 4, 3, 4, 6, 5, 3, 4, 3, 7, 2, 4, 6, 2],
       [4, 6, 2, 4, 2, 3, 6, 5, 2, 5, 6, 4, 5, 0, 7, 4],
       [4, 4, 4, 3, 4, 5, 3, 5, 4, 1, 5, 5, 2, 5, 4, 4]], dtype=uint8)